In [1]:
import sys
import os
import torch
from sklearn.metrics import r2_score
sys.path.insert(0, os.path.abspath(".."))
from slim_gsgp.datasets.data_loader import load_pandas_df
import pandas as pd
import numpy as np
from slim_gsgp.main_gp import gp
from slim_gsgp.main_slim import slim
from slim_gsgp.main_gsgp import gsgp
from slim_gsgp.utils.utils import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

from imblearn.over_sampling import SMOTENC, SMOTE

In [2]:
def print_scores(y_test, predictions):
    print("Roc Score:", roc_auc_score(y_test, predictions))
    print("F1 Score:", f1_score(y_test, predictions))
    print("Accuracy Score:", accuracy_score(y_test, predictions))

In [3]:
df = pd.read_csv('data/BankChurners.csv')
df.drop(columns=['CLIENTNUM', 'CLIENTNUM'], inplace=True)
df = df.iloc[:, :-2]
df.isna().sum()

Attrition_Flag              0
Customer_Age                0
Gender                      0
Dependent_count             0
Education_Level             0
Marital_Status              0
Income_Category             0
Card_Category               0
Months_on_book              0
Total_Relationship_Count    0
Months_Inactive_12_mon      0
Contacts_Count_12_mon       0
Credit_Limit                0
Total_Revolving_Bal         0
Avg_Open_To_Buy             0
Total_Amt_Chng_Q4_Q1        0
Total_Trans_Amt             0
Total_Trans_Ct              0
Total_Ct_Chng_Q4_Q1         0
Avg_Utilization_Ratio       0
dtype: int64

In [4]:
df['Attrition_Flag'] = df['Attrition_Flag'].map({'Existing Customer': 0, 'Attrited Customer': 1})
df = df.select_dtypes(include=np.number)
df = df[df.columns[1:].tolist() + [df.columns[0]]]
df

,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Attrition_Flag
0,45,3,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0
1,49,5,44,6,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0
2,51,3,36,4,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0
3,40,4,34,3,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0
4,40,3,21,5,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10122,50,2,40,3,2,3,4003.0,1851,2152.0,0.703,15476,117,0.857,0.462,0
10123,41,2,25,4,2,3,4277.0,2186,2091.0,0.804,8764,69,0.683,0.511,1
10124,44,1,36,5,3,4,5409.0,0,5409.0,0.819,10291,60,0.818,0.000,1
10125,30,2,36,4,3,3,5281.0,0,5281.0,0.535,8395,62,0.722,0.000,1


In [5]:
X, y = load_pandas_df(df, X_y=True)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, p_test=0.4)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, p_test=0.5)

In [26]:
slim_f1, slim_f1_pop = slim(X_train=X_train, y_train=y_train,
                  X_test=X_val, y_test=y_val,
                  dataset_name='ppb', slim_version='SLIM+SIG2', pop_size=200, n_iter=100, minimization=False, fitness_function='f1_score',
                  ms_lower=0, ms_upper=1, p_inflate=0.5)
predictions = slim_f1.predict(X_test)
predictions = torch.round(torch.sigmoid(predictions))
print_scores(y_test, predictions)

Verbose Reporter
-----------------------------------------------------------------------------------------------------------------------------------------
|         Dataset         |  Generation  |     Train Fitness     |       Test Fitness       |        Timing          |      Nodes       |
-----------------------------------------------------------------------------------------------------------------------------------------
|     ppb                 |       0      |   0.43488946557044983 |   0.44675925374031067    |   0.302293062210083    |      3           |
|     ppb                 |       1      |   0.43488946557044983 |   0.44675925374031067    |   0.3432340621948242   |      3           |
|     ppb                 |       2      |   0.4459990859031677  |   0.4536585509777069     |   0.3802764415740967   |      33          |
|     ppb                 |       3      |   0.4479999840259552  |   0.46211177110671997    |   0.4087989330291748   |      48          |
|     ppb        

In [ ]:
preds = []
for i in range(len(slim_f1_pop.population)):
    model = slim_f1_pop[i]
    model.version = slim_f1_pop.population[0].version
    pred = model.predict(X_train)
    #pred = torch.round(torch.sigmoid(pred))
    preds.append(pred)
    

from sklearn.linear_model import ElasticNet

lr = ElasticNet(l1_ratio=0.5, alpha=0.001)
lr.fit(np.array(preds).T, y_train)
stacked_pred = lr.predict(np.array(preds).T)
stacked_pred = torch.round(torch.sigmoid(torch.tensor(stacked_pred)))
print_scores(y_train, stacked_pred)

In [42]:
slim_f1_pop[100].fitness

tensor(0.4346)

Roc Score: 0.5950385418127354
F1 Score: 0.3171466845277964
Accuracy Score: 0.32894520322527565


c:\Users\leond\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.659e-01, tolerance: 8.097e-02
  model = cd_fast.enet_coordinate_descent(


In [38]:
X_train_over, y_train_over = SMOTE( random_state=42).fit_resample(X_train, y_train)
X_train_over = torch.tensor(X_train_over).float()
y_train_over = torch.tensor(y_train_over).float()

In [ ]:
slim_rmse = slim(X_train=X_train_over, y_train=y_train_over,
                  X_test=X_val, y_test=y_val,
                   slim_version='SLIM+SIG2', pop_size=200, n_iter=200, minimization=True, fitness_function='sigmoid_rmse',
                  ms_lower=0, ms_upper=1, p_inflate=0.5)

predictions = slim_rmse.predict(X_test)
predictions = torch.round(torch.sigmoid(predictions))
print_scores(y_test, predictions)

c:\Users\leond\OneDrive\Desktop\MasterThesis\slim\slim_gsgp\main_slim.py:165: UserWarning: No dataset name set. Using default value of dataset_1.
  warnings.warn("No dataset name set. Using default value of dataset_1.")


Verbose Reporter
-----------------------------------------------------------------------------------------------------------------------------------------
|         Dataset         |  Generation  |     Train Fitness     |       Test Fitness       |        Timing          |      Nodes       |
-----------------------------------------------------------------------------------------------------------------------------------------
|     dataset_1           |       0      |   0.4805372357368469  |   0.5270711779594421     |   0.3876817226409912   |      3           |
|     dataset_1           |       1      |   0.4805372357368469  |   0.5270711779594421     |   0.22804808616638184  |      3           |
|     dataset_1           |       2      |   0.4805372357368469  |   0.5270711779594421     |   0.20754718780517578  |      3           |
|     dataset_1           |       3      |   0.4667612612247467  |   0.48630011081695557    |   0.21622228622436523  |      16          |
|     dataset_1  

In [ ]:
gsgp_f1 = gsgp(X_train=X_train, y_train=y_train,
                  X_test=X_val, y_test=y_val, reconstruct=True, 
                  pop_size=200, n_iter=200, minimization=False, fitness_function='f1_score', seed = 0,
                  )

predictions = gsgp_f1.predict(X_test)
predictions = torch.round(torch.sigmoid(predictions))
print_scores(y_test, predictions)

c:\Users\leond\OneDrive\Desktop\MasterThesis\slim\slim_gsgp\main_gsgp.py:145: UserWarning: No dataset name set. Using default value of dataset_1.
  warnings.warn("No dataset name set. Using default value of dataset_1.")


Verbose Reporter
-----------------------------------------------------------------------------------------------------------------------------------------
|         Dataset         |  Generation  |     Train Fitness     |       Test Fitness       |        Timing          |      Nodes       |
-----------------------------------------------------------------------------------------------------------------------------------------
|     dataset_1           |       0      |   0.44249269366264343 |   0.4533648192882538     |   0.3840603828430176   |      15          |
|     dataset_1           |       1      |   0.44249269366264343 |   0.4533648192882538     |   0.3324120044708252   |      15          |
|     dataset_1           |       2      |   0.44922545552253723 |   0.4681372344493866     |   0.30107760429382324  |      23          |
|     dataset_1           |       3      |   0.4587652087211609  |   0.475347638130188      |   0.31645798683166504  |      25          |
|     dataset_1  

In [ ]:
gsgp_rmse = gsgp(X_train=X_train_over, y_train=y_train_over,
                  X_test=X_val, y_test=y_val, reconstruct=True, 
                  pop_size=200, n_iter=200, minimization=True, fitness_function='sigmoid_rmse', seed = 1,
                  )

predictions = gsgp_rmse.predict(X_test)
predictions = torch.round(torch.sigmoid(predictions))
print_scores(y_test, predictions)

c:\Users\leond\OneDrive\Desktop\MasterThesis\slim\slim_gsgp\main_gsgp.py:145: UserWarning: No dataset name set. Using default value of dataset_1.
  warnings.warn("No dataset name set. Using default value of dataset_1.")


Verbose Reporter
-----------------------------------------------------------------------------------------------------------------------------------------
|         Dataset         |  Generation  |     Train Fitness     |       Test Fitness       |        Timing          |      Nodes       |
-----------------------------------------------------------------------------------------------------------------------------------------
|     dataset_1           |       0      |   0.4804946482181549  |   0.5271535515785217     |   0.4003739356994629   |      3           |
|     dataset_1           |       1      |   0.47481948137283325 |   0.4962330758571625     |   0.2362966537475586   |      11          |
|     dataset_1           |       2      |   0.45946815609931946 |   0.46918341517448425    |   0.3241128921508789   |      17          |
|     dataset_1           |       3      |   0.45946815609931946 |   0.46918341517448425    |   0.22684502601623535  |      17          |
|     dataset_1  

In [ ]:
gp_f1 = gp(X_train=X_train, y_train=y_train, p_xo=0.2,
                  X_test=X_val, y_test=y_val, pop_size=200, n_iter=100, minimization=False, fitness_function='f1_score', seed = 0,
                  )
predictions = gp_f1.predict(X_test)
predictions = torch.round(torch.sigmoid(predictions))
print_scores(y_test, predictions)

Verbose Reporter
-----------------------------------------------------------------------------------------------------------------------------------------
|         Dataset         |  Generation  |     Train Fitness     |       Test Fitness       |        Timing          |      Nodes       |
-----------------------------------------------------------------------------------------------------------------------------------------
|     dataset_1           |       0      |   0.44525253772735596 |   0.45308926701545715    |   0.18932867050170898  |      5           |
|     dataset_1           |       1      |   0.44525253772735596 |   0.45308926701545715    |   0.2543361186981201   |      5           |
|     dataset_1           |       2      |   0.44525253772735596 |   0.45308926701545715    |   0.25008082389831543  |      5           |
|     dataset_1           |       3      |   0.44525253772735596 |   0.45308926701545715    |   0.31145787239074707  |      5           |
|     dataset_1  

In [ ]:
gp_rmse = gp(X_train=X_train_over, y_train=y_train_over,
                  X_test=X_val, y_test=y_val, pop_size=200, n_iter=100, minimization=True, fitness_function='sigmoid_rmse', seed = 0,
                  )
predictions = gp_rmse.predict(X_test)
predictions = torch.round(torch.sigmoid(predictions))
print_scores(y_test, predictions)

c:\Users\leond\OneDrive\Desktop\MasterThesis\slim\slim_gsgp\main_gp.py:148: UserWarning: No dataset name set. Using default value of dataset_1.
  warnings.warn("No dataset name set. Using default value of dataset_1.")


Verbose Reporter
-----------------------------------------------------------------------------------------------------------------------------------------
|         Dataset         |  Generation  |     Train Fitness     |       Test Fitness       |        Timing          |      Nodes       |
-----------------------------------------------------------------------------------------------------------------------------------------
|     dataset_1           |       0      |   0.47669273614883423 |   0.5389356017112732     |   0.18155336380004883  |      5           |
|     dataset_1           |       1      |   0.47669273614883423 |   0.5389356017112732     |   0.1466360092163086   |      5           |
|     dataset_1           |       2      |   0.47613951563835144 |   0.5272106528282166     |   0.14223527908325195  |      5           |
|     dataset_1           |       3      |   0.4737030863761902  |   0.5282784700393677     |   0.1464700698852539   |      5           |
|     dataset_1  

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
predictions = rf.predict(X_test)
print_scores(y_test, predictions)

Roc Score: 0.9102726928228659
F1 Score: 0.8764805414551607
Accuracy Score: 0.9639506172839506
